
# Decision Trees from Scratch  
### Entropy, Gini, Routing & Conceptual Foundations

This notebook contains a **complete, from-scratch implementation of a Decision Tree** for classification.
It is designed as **study notes + executable code**, preserving clarity and pedagogy.


## 1. Imports and Dataset

In [ ]:

import numpy as np
import pandas as pd


In [ ]:

data = {
    "Age": [22, 25, 47, 52, 46, 56, 48],
    "Salary": [25000, 32000, 60000, 70000, 65000, 80000, 62000],
    "Buy": ["No", "No", "Yes", "Yes", "Yes", "Yes", "Yes"]
}

df = pd.DataFrame(data)


## 2. Gini Impurity

In [ ]:

def gini(labels):
    values, counts = np.unique(labels, return_counts=True)
    probabilities = counts / counts.sum()
    return 1 - np.sum(probabilities ** 2)

gini(df["Buy"])


## 3. Entropy

In [ ]:

def entropy(labels):
    values, counts = np.unique(labels, return_counts=True)
    probabilities = counts / counts.sum()
    return -np.sum(probabilities * np.log2(probabilities))

entropy(df["Buy"])


## 4. Dataset Split

In [ ]:

def split_data(df, feature, threshold):
    left = df[df[feature] <= threshold]
    right = df[df[feature] > threshold]
    return left, right


## 5. Weighted Gini

In [ ]:

def weighted_gini(left, right, target):
    n = len(left) + len(right)
    return (
        len(left) / n * gini(left[target]) +
        len(right) / n * gini(right[target])
    )


## 6. Information Gain

In [ ]:

def information_gain(parent, left, right, target):
    parent_entropy = entropy(parent[target])
    n = len(left) + len(right)
    split_entropy = (
        len(left) / n * entropy(left[target]) +
        len(right) / n * entropy(right[target])
    )
    return parent_entropy - split_entropy


## 7. Best Split

In [ ]:

def best_split(df, target):
    best_feature = None
    best_threshold = None
    best_score = float("inf")

    for feature in df.columns:
        if feature == target:
            continue

        thresholds = df[feature].unique()
        for t in thresholds:
            left, right = split_data(df, feature, t)
            if len(left) == 0 or len(right) == 0:
                continue
            score = weighted_gini(left, right, target)
            if score < best_score:
                best_score = score
                best_feature = feature
                best_threshold = t

    return best_feature, best_threshold, best_score


## 8. Majority Class

In [ ]:

def majority_class(labels):
    return labels.value_counts().idxmax()


## 9. Tree Node

In [ ]:

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value


## 10. Build Tree

In [ ]:

def build_tree(df, target, depth=0, max_depth=3):
    labels = df[target]

    if len(labels.unique()) == 1:
        return Node(value=labels.iloc[0])

    if depth >= max_depth:
        return Node(value=majority_class(labels))

    feature, threshold, score = best_split(df, target)
    if feature is None:
        return Node(value=majority_class(labels))

    left_df, right_df = split_data(df, feature, threshold)
    left_child = build_tree(left_df, target, depth + 1, max_depth)
    right_child = build_tree(right_df, target, depth + 1, max_depth)

    return Node(feature, threshold, left_child, right_child)


## 11. Train Tree

In [ ]:

tree = build_tree(df, "Buy")


## 12. Prediction

In [ ]:

def predict(node, sample):
    if node.value is not None:
        return node.value

    if sample[node.feature] <= node.threshold:
        return predict(node.left, sample)
    else:
        return predict(node.right, sample)


In [ ]:

sample = {"Age": 30, "Salary": 40000}
predict(tree, sample)



## 13. Conceptual Bridge

Decision Trees → Routing Networks → Mixture of Experts  
Trees are the conceptual ancestors of modern sparse neural architectures.
